## LionAGI - 106


Planning & Pydantic Structured Output: Build your own self-optimizing ReAct agent (pt 1)

In [1]:
# %pip install lionagi

In [2]:
from lionagi import __version__
from lionfuncs import time

print("lionagi version \t", __version__)
print("last edited date \t", time(type_="iso", sep="T").split("T")[0])

lionagi version 	 0.3.5
last edited date 	 2024-10-13


In [3]:
from IPython.display import display, Markdown


def printmd(string):
    display(Markdown(string))


from pathlib import Path

datapath = Path.cwd() / "data" / "lion-core-source"

### Install LLamaIndex (RAG Tool provider)

In [4]:
# %pip install lionagi llama-index nbconvert

In [5]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-4o-mini")

docs = SimpleDirectoryReader(
    input_dir=datapath, recursive=True, required_exts=[".py", ".md", ".ipynb"]
).load_data()

index = VectorStoreIndex.from_documents(documents=docs)
query_engine = index.as_query_engine(llm=llm)

response = query_engine.query(
    "What is the difference between an lion-core Note and a dictionary??",
)

printmd(response.response)

The `Note` class is a flexible container specifically designed for managing nested dictionary data structures, allowing for complex and hierarchical information storage. In contrast, a dictionary is a standard data structure in Python that stores key-value pairs without the additional methods and functionalities provided by the `Note` class for manipulating and accessing nested data. Essentially, while a dictionary is a basic data structure, the `Note` class enhances this functionality with specialized methods tailored for more complex data management.

In [6]:
from lionfuncs import to_dict


async def query_lion_core(query: str):
    """
    Use this tool to query the lion-core repository for information.
    you should frequently refer to this if you are unsure about anything.

    Args:
        query: str: A question, term or query about lion-core.
    """
    response = await query_engine.aquery(query)
    return to_dict(
        response,
        recursive=True,
        recursive_python_only=False,
        max_recursive_depth=4,
    )

---

what is reason action? in terms of AI? And why is it useful?

In tutorial 105, we saw how to use a branch to do function calling. Function Calling is in essence, a type of structured output, and we can do these easier ourselves. 

Let's build our own reason action agent capable of planning using pydantic. A simple illustration of agentic research.

In [7]:
from lionfuncs import to_num
from pydantic import BaseModel, Field


class ReasonModel(BaseModel):
    title: str = Field(
        ...,
        title="Title",
        description="The title of the reason.",
    )
    content: str = Field(
        ...,
        title="Content",
        description="The content that supports or explains the reason.",
    )
    rating: float = Field(
        ...,
        title="Rating",
        description=(
            "The critical objective self rating (1-10) of the reason in terms of correctness."
            " >8 means can continue the path of reasoning, "
            "5-7 you need to recheck your reasoning and consider reversing to previously more confident reason path, "
            "below 5 means you need to stop, must mark more_reason_needed as False, and flag failed as True"
        ),
    )
    more_reason_needed: bool = Field(
        False,
        description=(
            "A flag to indicate whether there are more reasons needed to solve the task."
            "if you are confident that you have enough reasons, set this to False. and stop the reasoning."
        ),
    )

    def is_valid_reason(self):
        try:
            if to_num(self.rating) < 5:
                return False
            if self.content == "":
                return False
            if self.title == "":
                return False
            return True
        except Exception:
            return False

let us try it out

In [8]:
from lionagi import Branch, iModel

system_prompt = """
you are an expert in python and coding, you are asked to provide guidance on how to use the lion-core library.
you have access to the lion-core repository query engine, you can ask questions and get answers.
"""

instruction1 = """
From given context, reason the approach of querying the lion-core repository for information.
"""

In [10]:
imodel = iModel(model="o1-mini", temperature=1)
branch = Branch(imodel=imodel)

response = await branch.chat(
    instruction=system_prompt + instruction1,
    pydantic_model=ReasonModel,
    return_pydantic_model=True,
)

In [11]:
from lionfuncs import as_readable_json

str_ = as_readable_json(response)

print(str_)

{
    "title": "Approach for Querying the lion-core Repository",
    "content": "To effectively utilize the lion-core library, begin by accessing its repository query engine. Start by identifying the specific functionalities or modules you need assistance with. Formulate precise queries related to those areas, such as requesting examples, documentation references, or best practices. Analyze the responses to gain a deeper understanding of the library's capabilities and integration methods. This systematic querying approach will streamline your development process and ensure you leverage the lion-core library efficiently.",
    "rating": 9.0,
    "more_reason_needed": false
}


In [12]:
class QueryModel(BaseModel):
    function_name: str = Field(
        ...,
        title="Function Name",
        description="The name of the function to be called. Must pick from given options.",
    )
    query: str = Field(
        ...,
        title="Query",
        description="A detailed precise query to be made to a query engine bot with natural language interface.",
    )


class PlanModel(BaseModel):
    title: str = Field(
        ...,
        title="Title",
        description="The title of the plan.",
    )
    reasons: list[ReasonModel] = Field(
        ...,
        title="Reason",
        description="The reason for the plan.",
    )
    content: str = Field(
        ...,
        title="Content",
        description="The detailed plan content",
    )
    actions: list[QueryModel] = Field(
        default_factory=list,
        title="Actions",
        description="The actions to be taken., You must use action if you don't have information to provide evidence-backed information.",
    )

In [13]:
from lionfuncs import function_to_schema

instruction = """
perform agentic RAG, prepare step by query plan, you should provide 5-10 questions to ask our query engine, the questions should be logically sequenced and should help in reasoning the approach to build an agent using lion-core.
"""

response = await branch.chat(
    instruction=instruction,
    context={
        "tool_schema": function_to_schema(query_lion_core),
    },
    pydantic_model=PlanModel,
    return_pydantic_model=True,
    clear_messages=True,
)

str_ = as_readable_json(response)

print(str_)

{
    "title": "Step-by-Step Query Plan for Building an Agent using lion-core",
    "reasons": [
        {
            "title": "Understand Core Architecture",
            "content": "To build an effective agent, it's crucial to comprehend the fundamental architecture of lion-core, including its primary modules and their interactions.",
            "rating": 9.0,
            "more_reason_needed": false
        },
        {
            "title": "Identify Integration Points",
            "content": "Determining where and how to integrate the agent within lion-core ensures seamless functionality and leverages existing features.",
            "rating": 8.5,
            "more_reason_needed": false
        },
        {
            "title": "Determine Extension Capabilities",
            "content": "Knowing how lion-core can be extended or customized allows for tailoring the agent to specific requirements and use cases.",
            "rating": 8.0,
            "more_reason_needed": false
    

In [14]:
response.actions

[QueryModel(function_name='query_lion_core', query='What are the main modules and components of lion-core?'),
 QueryModel(function_name='query_lion_core', query='How can external agents be integrated with lion-core?'),
 QueryModel(function_name='query_lion_core', query='What customization and extension points does lion-core offer for developing agents?'),
 QueryModel(function_name='query_lion_core', query='What APIs are available in lion-core for agent communication?'),
 QueryModel(function_name='query_lion_core', query='What are the authentication and authorization protocols used by lion-core?'),
 QueryModel(function_name='query_lion_core', query='Can you provide details on the data models and schemas used in lion-core?'),
 QueryModel(function_name='query_lion_core', query='What are the best practices for error handling and logging in lion-core when developing an agent?'),
 QueryModel(function_name='query_lion_core', query='Are there any existing agents built with lion-core that can b

In [15]:
for idx, item in enumerate(response.actions):
    printmd(f"Query No. {idx + 1}: {item.query} \n --- \n")
    query_response = (await query_lion_core(item.query))["response"]
    printmd(query_response + "\n\n --- \n\n")

Query No. 1: What are the main modules and components of lion-core? 
 --- 


The main modules and components of Lion-Core include:

1. **Core Components**:
   - **Element**: A foundational class for creating modular and composable components.
   - **Node**: An extension of Element for creating interconnected components in a graph-like structure.

2. **Data Structures**:
   - **Pile**
   - **Progression**
   - **Flow**

3. **Communication**:
   - **Message System**
   - **Mail Manager**
   - **Exchange**: Manages asynchronous communication between components.

4. **Utility Systems**:
   - **Converter System**
   - **Form & Validator**
   - **Rule System**

5. **AI and ML Tools**:
   - **Action System**
   - **Session Management**

These components can be used independently or in combination, allowing for modular and scalable application development.

 --- 



Query No. 2: How can external agents be integrated with lion-core? 
 --- 


Integration of external agents with Lion-Core can be achieved through the framework's flexible architecture, which allows for the incorporation of various components and functionalities. By utilizing the existing classes and structures, such as `Node`, `Edge`, and `Pile`, external agents can interact with the graph system effectively. Additionally, leveraging community resources like GitHub Issues for feature requests and discussions on the Discord server can facilitate collaboration and support for integrating new functionalities.

 --- 



Query No. 3: What customization and extension points does lion-core offer for developing agents? 
 --- 


...

The provided information does not mention any existing agents built with Lion-Core. For inquiries about specific agents or examples, it may be helpful to check the community discussions or GitHub repository for any shared projects or references.

 --- 



Query No. 9: What testing frameworks are recommended for ensuring the reliability of an agent within lion-core? 
 --- 


The provided information does not specify any testing frameworks for ensuring the reliability of an agent within lion-core. For details on recommended testing frameworks, further resources or documentation specific to testing in the lion-core environment would be needed.

 --- 



Query No. 10: What performance optimization techniques are suggested for agents interacting with lion-core? 
 --- 


The provided information does not include specific performance optimization techniques for agents interacting with Lion-Core. It primarily focuses on community engagement, development goals, and the roadmap for enhancing the library. For detailed performance optimization techniques, further documentation or resources would be needed.

 --- 

